<a href="https://colab.research.google.com/github/Nuthanmc/Codsoft-Internship/blob/master/Codsoft_task_CreditCr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('/content/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [3]:
df.shape

(9965, 31)

In [4]:
df.dtypes

Time        int64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class     float64
dtype: object

In [5]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        1
V4        1
V5        1
V6        1
V7        1
V8        1
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [8]:
fraud_count = df["Class"].value_counts()
fraud_rate = 100*fraud_count/df.shape[0]
fraud_data = pd.concat([fraud_count,fraud_rate],axis=1).reset_index() # Use reset_index() to reset the index
fraud_data.columns = ['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,0.0,9926,99.608630
1,1.0,38,0.381335


In [9]:
df_fraud = df[df['Class'] == 1]
df_not_fraud = df[df['Class'] == 0]
df_not_fraud_sampled = df_not_fraud.sample(df_fraud.shape[0], replace=False, random_state=101) # Changed 'Flase' to 'False'

df_balanced = pd.concat([df_fraud,df_not_fraud_sampled],axis=0).sample(frac=1,replace=False,random_state=101).reset_index().drop('index',axis=1) # Changed 'Flase' to 'False'
df_balanced

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,4462,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,-1.525412,2.032912,-6.560124,0.022937,-1.470102,-0.698826,-2.282194,-4.781831,-2.615665,-1.334441,-0.430022,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1.0
1,14152,-4.710529,8.636214,-15.496222,10.313349,-4.351341,-3.322689,-10.788373,5.060381,-5.689311,-11.712187,11.152491,-16.558197,0.302645,-17.475921,-0.412393,-10.222203,-13.799148,-5.008585,1.162026,1.434240,1.990545,0.223785,0.554408,-1.204042,-0.450685,0.641836,1.605958,0.721644,1.00,1.0
2,8415,-0.251471,4.313523,-6.891438,6.796797,0.616297,-2.966327,-2.436653,0.489328,-3.371639,-6.810813,7.620089,-10.285283,-0.342444,-11.543498,-1.334988,-2.689284,-3.204383,0.086522,-1.314495,0.632710,0.536892,-0.546126,-0.605240,-0.263743,1.539916,0.523574,0.891025,0.572741,1.00,1.0
3,8169,0.857321,4.093912,-7.423894,7.380245,0.973366,-2.730762,-1.496497,0.543015,-2.351190,-3.944238,6.355078,-7.309748,0.748451,-9.057993,-0.648945,-1.073117,1.524501,1.831364,-0.089724,0.483303,0.375026,0.145400,0.240603,-0.234649,-1.004881,0.435832,0.618324,0.148469,1.00,1.0
4,11131,-1.426623,4.141986,-9.804103,6.666273,-4.749527,-2.073129,-10.089931,2.791345,-3.249516,-11.420451,10.853012,-15.969208,0.546690,-14.690729,0.912337,-12.227189,-18.587366,-6.920762,3.166999,1.410678,1.865679,0.407809,0.605809,-0.769348,-1.746337,0.502040,1.977258,0.711607,1.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2322,-0.353417,-3.421649,-0.253020,-1.052406,-2.055726,-0.205536,0.446410,-0.260456,2.740303,-2.230990,-0.947261,1.740290,1.061065,-0.635066,0.092974,-1.400804,0.384916,0.186129,1.366842,1.671162,0.528372,0.213120,-1.010238,0.190236,0.610824,-0.655040,-0.034321,0.172945,850.15,0.0
72,7610,0.725646,2.300894,-5.329976,4.007683,-1.730411,-1.732193,-3.968593,1.063728,-0.486097,-4.624985,5.588724,-7.148243,1.680451,-6.210258,0.495282,-3.599540,-4.830324,-0.649090,2.250123,0.504646,0.589669,0.109541,0.601045,-0.364700,-1.843078,0.351909,0.594550,0.099372,1.00,1.0
73,10965,1.228300,-0.066461,0.755817,0.135216,-0.631242,-0.333522,-0.557807,-0.026518,1.523648,-0.345559,2.020920,-1.870575,1.602886,1.683661,-0.079769,0.785979,-0.065912,0.776331,0.010697,-0.071843,-0.038679,0.109397,-0.065739,0.015735,0.215262,1.034781,-0.094485,-0.008213,14.95,0.0
74,2248,0.788193,-0.638952,-2.003211,0.276192,2.296666,3.193820,0.340287,0.580790,-0.497168,0.033861,-0.156600,-0.005996,-0.066776,0.778227,0.873080,0.191328,-0.814892,0.132254,-0.023551,0.532179,0.117398,-0.399349,-0.424014,1.022990,0.900380,-0.294534,-0.060297,0.052761,281.15,0.0


In [10]:
fraud_count = df_balanced["Class"].value_counts()
fraud_rate = 100*fraud_count/df_balanced.shape[0]
fraud_data = pd.concat([fraud_count,fraud_rate],axis=1).reset_index() # Use reset_index() to reset the index
fraud_data.columns = ['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,1.0,38,50.0
1,0.0,38,50.0


In [11]:
X_train,X_test,y_train,y_test = train_test_split(df_balanced.drop('Class',axis=1),df_balanced['Class'],test_size=0.2,random_state=101)

print(f'''X_train: {X_train.shape}
X_test: {X_test.shape}
y_train: {y_train.shape}
y_test: {y_test.shape}''')

X_train: (60, 30)
X_test: (16, 30)
y_train: (60,)
y_test: (16,)


In [12]:
randomForestModel = Pipeline([
    ('scaler',StandardScaler()),
    ('classifier',RandomForestClassifier())
])

randomForestModel.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier())])

In [13]:
y_pred_logis = randomForestModel.predict(X_test)
y_pred_logis

array([1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1.])

In [14]:
cr = classification_report(y_test, y_pred_logis)
print(cr)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         6
         1.0       1.00      1.00      1.00        10

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [15]:
with open('./model.pkl', 'wb') as fp:
  pickle.dump(randomForestModel,fp)